In [5]:
cd '/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_three/dw_matrix_road_sign'

/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_three/dw_matrix_road_sign


In [6]:
!pip install hyperopt

In [7]:
import pandas as pd
import numpy as np
import os
import datetime

import matplotlib.pyplot as plt
from skimage import color, exposure

from sklearn.metrics import accuracy_score

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical

%load_ext tensorboard

from hyperopt import hp, STATUS_OK, tpe, Trials, fmin

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [0]:
train = pd.read_pickle('data/train.p')
test = pd.read_pickle('data/test.p')

X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

In [0]:
# vector to matix
if y_train.ndim == 1: y_train = to_categorical(y_train)
if y_test.ndim == 1: y_test = to_categorical(y_test)

In [0]:
input_shape = X_train.shape[1:]
num_classes = y_train.shape[1]

In [0]:
def train_model(model, X_train, y_train, params_fit={}):
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
  
  logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
  tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)
  
  model.fit(
      X_train,
      y_train,
      batch_size=params_fit.get('batch_size', 128),
      epochs=params_fit.get('epochs', 5),
      verbose=params_fit.get('verbose', 1),
      validation_data=params_fit.get('validation_data', (X_train, y_train)),
      callbacks=[tensorboard_callback]
            ) 
  return model

In [0]:
def predict(model_trained, X_test, y_test, scoring =accuracy_score):  
  y_test_norm = np.argmax(y_test, axis=1)

  y_pred_prob = model_trained.predict(X_test)

  # zwraca index, pod którym znajduje się najwyższa wartość prawdopodobieństwa
  y_pred = np.argmax(y_pred_prob, axis=1)

  return scoring(y_test_norm, y_pred)

In [0]:
def get_cnn_v5(input_shape, num_classes):
  return Sequential([
      Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=input_shape),
      Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'),
      MaxPool2D(),
      Dropout(0.3),

      Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
      Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
      MaxPool2D(),
      Dropout(0.3),

      Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
      Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
      MaxPool2D(),
      Dropout(0.3),

      Flatten(),

      Dense(1024, activation='relu'),
      Dropout(0.3),
      
      Dense(1024, activation='relu'),
      Dropout(0.3),

      Dense(num_classes, activation='softmax'),
  ])

In [14]:
model = get_cnn_v5(input_shape, num_classes)
model_trained = train_model(model, X_train, y_train)
predict(model_trained, X_test, y_test)

Epoch 1/5
272/272 [==============================] - 7s 24ms/step - loss: 2.3826 - accuracy: 0.3247 - val_loss: 0.7077 - val_accuracy: 0.7780
Epoch 2/5
272/272 [==============================] - 6s 22ms/step - loss: 0.6348 - accuracy: 0.7974 - val_loss: 0.1519 - val_accuracy: 0.9563
Epoch 3/5
272/272 [==============================] - 6s 22ms/step - loss: 0.3038 - accuracy: 0.9085 - val_loss: 0.0966 - val_accuracy: 0.9729
Epoch 4/5
272/272 [==============================] - 6s 22ms/step - loss: 0.2145 - accuracy: 0.9361 - val_loss: 0.0512 - val_accuracy: 0.9864
Epoch 5/5
272/272 [==============================] - 6s 22ms/step - loss: 0.1692 - accuracy: 0.9519 - val_loss: 0.0307 - val_accuracy: 0.9920


0.9689342403628118

In [15]:
model_trained.evaluate(X_test, y_test)[1]

138/138 [==============================] - 0s 3ms/step - loss: 0.1288 - accuracy: 0.9689


0.9689342379570007

In [0]:
def get_model(params):
  return Sequential([
      Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=input_shape),
      Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'),
      MaxPool2D(),
      Dropout(params['dropout_cnn_block_one']),

      Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
      Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
      MaxPool2D(),
      Dropout(params['dropout_cnn_block_two']),

      Conv2D(filters=128, kernel_size=(3,3), activation='relu', padding='same'),
      Conv2D(filters=128, kernel_size=(3,3), activation='relu'),
      MaxPool2D(),
      Dropout(params['dropout_cnn_block_three']),

      Flatten(),

      Dense(1024, activation='relu'),
      Dropout(params['dropout_dense_block_one']),

      Dense(1024, activation='relu'),
      Dropout(params['dropout_dense_block_two']),

      Dense(num_classes, activation='softmax'),
  ])

In [0]:
def func_obj(params):
  model = get_model(params)
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

  model.fit(
      X_train,
      y_train,
      batch_size=int(params.get('batch_size', 128)),
      epochs=5,
      verbose=0
      ) 
  score = model.evaluate(X_test, y_test, verbose=0)
  accuracy = score[1]
  print(params, 'accuracy={}'.format(accuracy))

  return {'loss': -accuracy, 'status': STATUS_OK, 'model': model}

In [25]:
space = {
    'batch_size': hp.quniform('batch_size', 100, 200, 10),
    'dropout_cnn_block_one': hp.uniform('dropout_cnn_block_one', 0.3, 0.5),
    'dropout_cnn_block_two': hp.uniform('dropout_cnn_block_two', 0.3, 0.5),
    'dropout_cnn_block_three': hp.uniform('dropout_cnn_block_three', 0.3, 0.5),
    
    'dropout_dense_block_one': hp.uniform('dropout_dense_block_one', 0.4, 0.7),
    'dropout_dense_block_two': hp.uniform('dropout_dense_block_two', 0.4, 0.7)
}

best = fmin(
    func_obj,
    space,
    tpe.suggest,
    30,
    Trials()
)

{'batch_size': 110.0, 'dropout_cnn_block_one': 0.3136144873654729, 'dropout_cnn_block_three': 0.3311990316634435, 'dropout_cnn_block_two': 0.4670175419097907, 'dropout_dense_block_one': 0.5028871630177582, 'dropout_dense_block_two': 0.44807103579229696}
accuracy=0.9657596349716187
{'batch_size': 150.0, 'dropout_cnn_block_one': 0.36521230396455334, 'dropout_cnn_block_three': 0.3646086629668792, 'dropout_cnn_block_two': 0.3033183393001476, 'dropout_dense_block_one': 0.4462983462226734, 'dropout_dense_block_two': 0.5116475556948596}
accuracy=0.961904764175415
{'batch_size': 200.0, 'dropout_cnn_block_one': 0.43569770646673855, 'dropout_cnn_block_three': 0.41972604173994615, 'dropout_cnn_block_two': 0.3088681572180006, 'dropout_dense_block_one': 0.5121412038381723, 'dropout_dense_block_two': 0.6107594451898839}
accuracy=0.866439938545227
{'batch_size': 150.0, 'dropout_cnn_block_one': 0.4514636395023499, 'dropout_cnn_block_three': 0.4989475978556634, 'dropout_cnn_block_two': 0.38855383251745